# Language-Recognition using ConvNets

_written by Joscha S. Rieber (Fraunhofer IAIS) in 2020_

## Dataset preparation

Please go to the [Mozilla Common Voice Website](https://voice.mozilla.org) and download the full German and English datasets. In the following scripts we will thin out the datasets to make them more handy and play with the data.
* Download German and English datasets
* Extract them
* Define paths below

In [1]:
train = 'train'
test = 'test'

eng = 'english'
ger = 'german'

languages = [eng, ger]
categories = [train, test]

original_dataset_paths = {}

original_dataset_paths[eng] = '/home/jrieber/Downloads/en/clips/' # TODO: Adapt this folder!
original_dataset_paths[ger] = '/home/jrieber/Downloads/de/clips/' # TODO: Adapt this folder!

target_root_path = '../data/'

num_files_to_take_for_each_language = 200000#100000#3000
train_rate = 0.8  # Use 80 % of the data for training and the rest for testing

### Check paths

If something goes wrong here, check paths again and read the documentation of the GitHub repository and check how to set-up your environment correctly

In [2]:
import os

for lang in languages:
    if not os.path.isdir(original_dataset_paths[lang]):
        raise
    for category in categories:
        if not os.path.isdir(target_root_path + category + '/' + lang):
            raise

### Collect only num_files_to_take_for_each_language files which duration is between 5 and 10 seconds

In [3]:
# If this goes wrong, check your environment and read the documentation

import librosa as lr
from glob import glob
from random import shuffle
from shutil import copy2
import numpy as np

In [ ]:
import warnings
warnings.simplefilter('ignore', UserWarning)

for lang in languages:
    
    print('')
    print('Copying files for language ' + lang + '...')
    print('')
    
    all_files = glob(original_dataset_paths[lang] + '*.mp3')
    shuffle(all_files)
    
    counter = 0
    
    category = train
    
    for file in all_files:
        try:
            audio_segment, sample_rate = lr.load(file)
            if np.count_nonzero(audio_segment) == 0:
                raise Exception('Audio is silent!')
            if audio_segment.ndim != 1:
                raise Exception('Audio signal has wrong number of dimensions: ' + str(audio_segment.ndim))
            duration_sec = lr.core.get_duration(audio_segment, sr=sample_rate)
        except Exception as e:
            print('WARNING! Error while loading file \"' + file + '\": ' + str(e) + ' - Skipping...')
            continue
        
        # Only copy audio files with a certain duration
        if 5.0 < duration_sec < 10.0:
            copy2(file, target_root_path + category + '/' + lang)
            counter += 1
        
        # Stop after collecting enough files
        if counter == int(num_files_to_take_for_each_language * train_rate):
            category = test
        if counter == num_files_to_take_for_each_language:
            break


Copying files for language english...

WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_533362.mp3":  - Skipping...
WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_85637.mp3": Audio is silent! - Skipping...
WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_581966.mp3": Audio is silent! - Skipping...
WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_592403.mp3": Audio is silent! - Skipping...
WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_542616.mp3":  - Skipping...
WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_581832.mp3": Audio is silent! - Skipping...
WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_675862.mp3": Audio is silent! - Skipping...
WARNING! Error while loading file "/home/jrieber/Downloads/en/clips/common_voice_en_507971.mp3": 

### Check number of collected files

In [ ]:
for category in categories:
    
    if category == train:
        num_files = int(num_files_to_take_for_each_language * train_rate)
    else:
        num_files = int(num_files_to_take_for_each_language * (1.0 - train_rate))
        
    for lang in languages:
        folder = target_root_path + category + '/' + lang + '/'
        all_files = glob(folder + '*.mp3')
        
        if len(all_files) < (num_files - 1):
            raise Exception('Folder \"' + folder + '\" only contains ' + str(len(all_files)) + ' files instead of ' + str(num_files) + '!')
            
print('Okay!')

Now make yourself familiar with the dataset by listening to some of the files